In [ ]:
# 필요한 라이브러리 설치
# !pip install -qU \
# sentence-transformers langchain langchain-openai langchain-community langchain-experimental \
# langchain-text-splitters tiktoken faiss-cpu openai pypdf requests pyvis\
# llama_index neo4j llama-index-graph-stores-neo4j llama-index-llms-langchain llama-index-embeddings-langchain \

# LangChain/LlamaIndex를 활용한 Graph RAG 구현 실습

## 목표
비정형 텍스트에서 **지식 그래프(노드, 관계)** 를 자동으로 추출하고, 이를 **VectorDB와 결합한 Graph RAG**를 구현하여 엔티티(개체) 간의 관계를 묻는 질문에 답하는 능력을 확보합니다.


In [ ]:
import os

from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

# LLM 및 Embedding 모델 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## OPENAI 임베딩 모델이 있지만 비용 문제로, 다른 모델 선택.
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#  한국어 특화 SentencePiece 기반 임베딩 모델 (SemanticChunker 및 VectorStore 생성에 사용)
model_name = "jhgan/ko-sroberta-multitask"
embed_model = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipykernel_64575/47291940.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
/tmp/ipykernel_64575/47291940.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name=model_name)


## 샘플 문서 로드
비교를 위해 의미 있는 내용이 충분히 담긴 문서를 로드합니다.  
여기서는 **한국지능정보사회진흥원**의 AI국가전략 보고서 중 "전국민 AI 일상화 실행 계획"을 사용하겠습니다.
 - https://www.nia.or.kr/site/nia_kor/ex/bbs/View.do?cbIdx=99952&bcIdx=27378&parentSeq=27378

In [9]:
import requests

# PDF 다운로드 URL
pdf_url = "https://www.nia.or.kr/common/board/Download.do?bcIdx=27378&cbIdx=99952&fileNo=1"
pdf_filename = "National_AI_Plan.pdf"

# 파일 저장
with open(pdf_filename, "wb") as f:
    f.write(requests.get(pdf_url).content)

print("PDF 파일 저장 완료:", pdf_filename)

PDF 파일 저장 완료: National_AI_Plan.pdf


In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("National_AI_Plan.pdf")
documents = loader.load()
print(documents[15].page_content[:500])

-6-
 Ⅲ. 국내 현황 진단◈전 세계적 디지털 모범국가로 나아가기 위한 정부의 전략 추진과민간의 선제적 혁신으로 전국민 AI일상화초석 마련○(정부)디지털 심화 시대에 대응한 신속한 전략･
정책추진을 통해 AI기술･
산업 경쟁력을 강화하고 민간 혁신을 뒷받침  * 대한민국 디지털 전략(‘22.9) → AI일상화·산업 고도화계획(’23.1)→초거대 AI경쟁력강화방안(‘23.4)-｢
AI법제정비단(’23.8~)｣
를 통해 AI확산에 따른 사회적 이슈를 정비하고,AI거버넌스 정립과 공동 번영을 위한 글로벌 협력 본격화○(민간)독자적 초거대 AI개발･
출시 본격화,중소･
스타트업 중심의다양한 응용서비스출시 등 산업 생태계 조성시작네이버, ‘클로바’LG, ‘엑사원’카카오, KoGPT SKT, ‘에이닷’KT, ‘믿음’
-의료 등 다양한 도메인에서의중소·스타트업 역시 초거대 AI기반전문서비스를 출시,체감되는 AI활용·확산을 주도
* 의료 AI 기업 ‘루닛’이 美정부의 암정복 프로젝트 ‘캔서 문샷


In [13]:
from llama_index.core import Document

# langchain 문서 → llama_index Document 변환
llama_documents = [Document(text=doc.page_content) for doc in documents]

## 지식 그래프 자동 추출 (Neo4j 연동)

LLM을 활용하여 문서에서 **(주체, 관계, 대상)** 형태의 트리플(Triplet)을 자동으로 추출하고, 이를 기반으로 지식 그래프를 생성합니다.   
이번 섹션에서는 Neo4j 그래프 데이터베이스에 지식 그래프를 저장하도록 설정합니다.

***Neo4j AuraDB:***
클라우드 기반의 Neo4j 서비스인 AuraDB를 사용합니다. 무료 티어가 제공되며, 연결 정보를 쉽게 얻을 수 있습니다.  

https://neo4j.com/product/auradb/

1. 해당 링크를 통해 회원 가입
2. free instance 만들기 (몇 분 정도 소요됩니다.)
3. password 꼭 ***따로 저장하기!*** - 해당 instance에 대한 password는 한번만 보여주므로, 따로 저장해서 잘 보관해야합니다.

아래 코드의 `NEO4J_URI`, `NEO4J_USERNAME`, `NEO4J_PASSWORD`에 실제 Neo4j 연결 정보를 입력하세요.


In [ ]:
# Neo4j AuraDB 연결 정보 설정 (!!! 실제 정보로 변경하세요 !!!)
NEO4J_URI = (
    "neo4j+s://AuraDB_ID.databases.neo4j.io"  # AuraDB URI, AuraDB_ID 부분에 b로 시작하는 instance ID를 입력하세요
)
NEO4J_USERNAME = "neo4j"  # AuraDB USERNAME
NEO4J_PASSWORD = "YOUR_AuraDB_PASSWORD"  # AuraDB 비밀번호

In [ ]:
from llama_index.core import Settings, StorageContext
from llama_index.core.indices.knowledge_graph import KnowledgeGraphIndex
from llama_index.graph_stores.neo4j import Neo4jGraphStore  # Neo4jGraphStore 임포트
from neo4j import GraphDatabase

# Neo4j 드라이버 연결 테스트
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    driver.verify_connectivity()
    print("Neo4j 데이터베이스에 성공적으로 연결되었습니다.")
    neo4j_connected = True
except Exception as e:
    print(f"Neo4j 데이터베이스 연결 실패: {e}")
    print("Neo4j가 실행 중이고, 연결 정보(URI, 사용자명, 비밀번호)가 올바른지 확인하세요.")
    print("연결에 실패하면 인메모리 그래프 스토어를 사용합니다.")
    neo4j_connected = False

# LlamaIndex 전체 설정 반영
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

# 그래프 스토어 초기화
if neo4j_connected:
    # Neo4jGraphStore 사용
    graph_store = Neo4jGraphStore(
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        database="neo4j",  # 또는 사용할 데이터베이스 이름
    )
    print("Neo4jGraphStore를 사용하도록 설정되었습니다.")
else:
    # Neo4j 연결 실패 시 인메모리 SimpleGraphStore 사용 (대체)
    graph_store = SimpleGraphStore()
    print("Neo4j 연결 실패로 인해 SimpleGraphStore (인메모리)를 사용합니다.")

storage_context = StorageContext.from_defaults(graph_store=graph_store)

if documents:  # 문서가 성공적으로 로드된 경우에만 지식 그래프 생성
    print("지식 그래프 추출 및 인덱싱을 시작합니다. (시간이 약 10분 정도 소요될 수 있습니다.)")

    # KnowledgeGraphIndex 생성
    # LLM이 문서를 읽고 트리플(주체, 관계, 대상)을 추출하여 그래프를 구축합니다.
    kg_index = KnowledgeGraphIndex.from_documents(
        llama_documents,
        storage_context=storage_context,
        include_text=True,  # 노드에 원본 텍스트 포함
        max_triplets_per_chunk=10,  # 청크당 최대 10개의 트리플 추출
        kg_extract_k=5,  # 지식 그래프에서 검색할 상위 5개 노드 (쿼리 시 사용)
        show_progress=True,  # 진행 상황 표시
    )

    print("\n지식 그래프 추출 및 인덱싱 완료.")
    if neo4j_connected:
        print("추출된 지식 그래프가 Neo4j 데이터베이스에 저장되었습니다.")
else:
    print("문서가 로드되지 않아 지식 그래프를 생성할 수 없습니다. PDF 로드 오류를 해결하세요.")
    kg_index = None  # 지식 그래프 인덱스를 None으로 설정하여 다음 단계에서 오류 방지

Neo4j 데이터베이스에 성공적으로 연결되었습니다.
Neo4jGraphStore를 사용하도록 설정되었습니다.
지식 그래프 추출 및 인덱싱을 시작합니다. (시간이 다소 소요될 수 있습니다.)


Processing nodes: 100%|██████████| 104/104 [09:25<00:00,  5.44s/it]


지식 그래프 추출 및 인덱싱 완료.
추출된 지식 그래프가 Neo4j 데이터베이스에 저장되었습니다.


## 지식 그래프 시각화

Neo4j에 저장된 데이터에서 쿼리로 노드와 관계를 직접 조회해 간단한 네트워크 그래프로 시각화하여, 데이터가 어떻게 구조화되었는지 눈으로 확인합니다.  

In [ ]:
import pandas as pd
from neo4j import GraphDatabase
from pyvis.network import Network

# Neo4j 드라이버 초기화
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))


def get_nodes_and_relationships(tx):
    # 모든 노드와 그 노드의 이름(id)를 가져옵니다.
    nodes = {}
    for record in tx.run("MATCH (n) RETURN id(n) as id, labels(n) as labels, properties(n) as props"):
        nodes[record["id"]] = {
            "labels": record["labels"],
            "props": record["props"],
            "label": record["props"].get("name") or record["props"].get("id") or str(record["id"]),
        }

    # 모든 관계(엣지)를 가져옵니다.
    relationships = []
    for record in tx.run("MATCH (s)-[r]->(o) RETURN id(s) as sid, type(r) as type, id(o) as oid"):
        relationships.append({"source": record["sid"], "target": record["oid"], "label": record["type"]})

    return nodes, relationships


with driver.session() as session:
    nodes, relationships = session.read_transaction(get_nodes_and_relationships)

# DataFrame으로 변환
nodes_df = pd.DataFrame([{"id": k, "label": v["label"]} for k, v in nodes.items()])
edges_df = pd.DataFrame(relationships)

if not nodes_df.empty and not edges_df.empty:
    net = Network(
        notebook=True, height="750px", width="100%", bgcolor="#222222", font_color="white", cdn_resources="remote"
    )

    # 노드 추가
    for _, node in nodes_df.iterrows():
        net.add_node(node["id"], label=node["label"], title=node["label"], color="#ADD8E6")

    # 엣지 추가
    for _, edge in edges_df.iterrows():
        net.add_edge(edge["source"], edge["target"], title=edge["label"], label=edge["label"], color="#FFD700")

    # 그래프 레이아웃 설정
    net.set_options(
        """
    var options = {
      "physics": {
        "enabled": true,
        "barnesHut": {
          "gravitationalConstant": -2000,
          "centralGravity": 0.3,
          "springLength": 95,
          "springConstant": 0.04,
          "damping": 0.09,
          "avoidOverlap": 0.5
        },
        "solver": "barnesHut"
      }
    }
    """
    )

    graph_html_path = "apollo_kg_visualization.html"
    net.save_graph(graph_html_path)
    print(f"\n지식 그래프 시각화가 '{graph_html_path}' 파일로 저장되었습니다.")

    # 1. Jupyter 노트북 환경에서 inline(iframe)으로 시도
    try:
        from IPython.display import IFrame, display

        display(IFrame(src=graph_html_path, width="100%", height="750"))
        print("Jupyter에서 바로 안 보인다면, 아래 안내에 따라 직접 파일을 열어보세요.")
    except Exception as e:
        print(f"IFrame 렌더링 중 오류 발생: {e}")

    # 2. 브라우저로 직접 열기(로컬에서도 확실)
    import os
    import webbrowser

    file_path = os.path.abspath(graph_html_path)
    print(f"브라우저에서 그래프 시각화를 보려면 아래 경로를 복사해서 주소창에 붙여넣으세요:\nfile://{file_path}")
    try:
        webbrowser.open(f"file://{file_path}")
    except Exception as e:
        print(f"웹 브라우저 자동 열기 중 오류: {e}")

else:
    print("노드 또는 엣지 데이터가 없어 시각화할 수 없습니다.")

/tmp/ipykernel_2556/2771204364.py:30: DeprecationWarning: read_transaction has been renamed to execute_read
  nodes, relationships = session.read_transaction(get_nodes_and_relationships)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 1, column: 18, offset: 17} for query: 'MATCH (n) RETURN id(n) as id, labels(n) as labels, properties(n) as props'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId


지식 그래프 시각화가 'apollo_kg_visualization.html' 파일로 저장되었습니다.


Jupyter에서 바로 안 보인다면, 아래 안내에 따라 직접 파일을 열어보세요.
브라우저에서 그래프 시각화를 보려면 아래 경로를 복사해서 주소창에 붙여넣으세요:
file:///home/downtown/aiffel/RAG/apollo_kg_visualization.html


gio: file:///home/downtown/aiffel/RAG/apollo_kg_visualization.html: Failed to find default application for content type ‘text/html’


In [33]:
# 아래 코드를 코랩 환경에서 실행하면 그래프 시각화를 바로 볼 수 있어요!

# from neo4j import GraphDatabase
# import pandas as pd
# from pyvis.network import Network

# # Neo4j 드라이버 초기화
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# def get_nodes_and_relationships(tx):
#     # 모든 노드와 그 노드의 이름(id)를 가져옵니다.
#     nodes = {}
#     for record in tx.run("MATCH (n) RETURN id(n) as id, labels(n) as labels, properties(n) as props"):
#         nodes[record["id"]] = {
#             "labels": record["labels"],
#             "props": record["props"],
#             "label": record["props"].get("name") or record["props"].get("id") or str(record["id"]),
#         }

#     # 모든 관계(엣지)를 가져옵니다.
#     relationships = []
#     for record in tx.run("MATCH (s)-[r]->(o) RETURN id(s) as sid, type(r) as type, id(o) as oid"):
#         relationships.append({
#             "source": record["sid"],
#             "target": record["oid"],
#             "label": record["type"]
#         })

#     return nodes, relationships

# with driver.session() as session:
#     nodes, relationships = session.read_transaction(get_nodes_and_relationships)

# # DataFrame으로 변환
# nodes_df = pd.DataFrame([{"id": k, "label": v["label"]} for k, v in nodes.items()])
# edges_df = pd.DataFrame(relationships)

# if not nodes_df.empty and not edges_df.empty:
#     net = Network(notebook=True, height="750px", width="100%", bgcolor="#222222", font_color="white", cdn_resources='remote')

#     # 노드 추가
#     for _, node in nodes_df.iterrows():
#         net.add_node(node['id'], label=node['label'], title=node['label'], color="#ADD8E6")

#     # 엣지 추가
#     for _, edge in edges_df.iterrows():
#         net.add_edge(edge['source'], edge['target'], title=edge['label'], label=edge['label'], color="#FFD700")

#     # 그래프 레이아웃 설정
#     net.set_options("""
#     var options = {
#       "physics": {
#         "enabled": true,
#         "barnesHut": {
#           "gravitationalConstant": -2000,
#           "centralGravity": 0.3,
#           "springLength": 95,
#           "springConstant": 0.04,
#           "damping": 0.09,
#           "avoidOverlap": 0.5
#         },
#         "solver": "barnesHut"
#       }
#     }
#     """)

#     graph_html_path = "apollo_kg_visualization.html"
#     net.save_graph(graph_html_path)
#     print(f"\n지식 그래프 시각화가 '{graph_html_path}' 파일로 저장되었습니다.")

#     from IPython.display import HTML, display
#     try:
#         display(HTML(filename=graph_html_path))
#     except Exception as e:
#         print(f"HTML 렌더링 중 오류 발생: {e}")
# else:
#     print("노드 또는 엣지 데이터가 없어 시각화할 수 없습니다.")

## Graph RAG 쿼리 엔진 구축 및 성능 테스트

추출된 지식 그래프를 기반으로 Graph RAG 쿼리 엔진을 생성하고, 엔티티 간의 명시적인 관계를 묻는 질문에 대해 정확한 답변을 얻는지 확인합니다.

In [31]:
if kg_index:  # 지식 그래프 인덱스가 성공적으로 생성된 경우에만 쿼리 엔진 생성
    # KnowledgeGraphIndex에서 쿼리 엔진 생성
    graph_query_engine = kg_index.as_query_engine(
        retriever_mode="keyword",
        with_hybrid=False,
        llm=llm,
    )

    print("\nGraph RAG 쿼리 엔진이 준비되었습니다.")

    # 성능 테스트 질문
    questions_graph_rag = [
        "AI 기반 복지 서비스의 도입 시기는 언제인가요?",
        "AI 디지털 교과서는 언제 도입될 예정인가요?",
        "AI 정수장 구축은 어떤 기관에서 검토하고 있나요?",
    ]

    print("\n--- Graph RAG 쿼리 엔진 테스트 ---")
    for i, q in enumerate(questions_graph_rag):
        print(f"\n[질문 {i + 1}] {q}")
        response = graph_query_engine.query(q)
        print(f"  [답변] {response.response}")
        print(f"  [소스 노드] {response.source_nodes}")  # 어떤 노드를 참조했는지 확인 가능
else:
    print("지식 그래프 인덱스가 생성되지 않아 Graph RAG 쿼리 엔진을 생성할 수 없습니다.")


Graph RAG 쿼리 엔진이 준비되었습니다.

--- Graph RAG 쿼리 엔진 테스트 ---

[질문 1] AI 기반 복지 서비스의 도입 시기는 언제인가요?
  [답변] AI 기반 복지 서비스의 도입 시기는 2023년으로 예정되어 있습니다.
  [소스 노드] [NodeWithScore(node=TextNode(id_='da42df6e-78d7-4292-a1ef-85e88850c165', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6587230b-a3e8-47c5-af23-f39720ec782f', node_type='4', metadata={}, hash='9ff27809f897a2ab3f39f6953ae6cef0514354e7edecfed79a207e4a2335583d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='a7719f52-b147-42db-a199-f489919b974a', node_type='1', metadata={}, hash='b0da0e5ff31f13e0aa7fe428746761c72d410389ed10d5a253240fd1e54add8f')}, metadata_template='{key}: {value}', metadata_separator='\n', text='AI이 신규 적용 및 확대가 필요하다고 생각하는분야로 ①주거편의(24.6%),②교육·학습(19.8%),③교통(10.1%)을 꼽음     * (주거편의) AI 홈 제품･서비스 등, (교육･학습) 영어발음 인식･회화, 수학수준 진단･강의 등 ☞ 따라서, 효용이 높거나 AI 확대 수요가 있는 분야를 중심으로 AI 이용 경험을 넓히는 “AI 일상화”의 확대 필요※ (대상

## Vector + Graph 하이브리드 검색

Vector RAG와 Graph RAG를 결합하여, 일반적인 질문은 VectorDB에서 답을 찾고, 관계에 대한 질문은 지식 그래프에서 답을 찾도록 하는 하이브리드 쿼리 엔진을 구성합니다.

In [32]:
from llama_index.core import VectorStoreIndex  # 올바른 임포트 경로

if documents and kg_index:  # 문서와 지식 그래프 인덱스가 모두 성공적으로 생성된 경우에만 하이브리드 검색
    # VectorStoreIndex 생성 (원본 문서에 대한 벡터 인덱스)
    vector_index = VectorStoreIndex.from_documents(
        llama_documents,
    )

    # KnowledgeGraphIndex 기반 쿼리 엔진 생성, 하이브리드 모드 활성화 및 벡터 인덱스 연결
    hybrid_query_engine = kg_index.as_query_engine(
        retriever_mode="keyword",
        with_hybrid=True,  # 하이브리드 모드 활성화
        vector_store_index=vector_index,  # 벡터 인덱스 연결
        alpha=0.5,  # 벡터 검색과 그래프 검색의 가중치 (0.5는 동일 가중치)
        llm=llm,
    )

    print("\n하이브리드 (Vector + Graph) 쿼리 엔진이 준비되었습니다.")

    # 하이브리드 검색 성능 테스트 질문
    questions_hybrid_rag = [
        "전국민 AI 일상화를 위한 핵심 추진 과제는 무엇이며, 각 과제와 관련된 주요 서비스나 사업은 무엇인가요?",  # 일반 정보 + 관계 확인 (하이브리드형)
        "보고서에 언급된 '사회적 약자 지원'과 관련된 AI 서비스들은 서로 어떻게 연관되어 있으며, 어떤 부처들이 협력하여 추진하나요?",  # 복합 관계 추론
        "'디지털플랫폼정부위원회'는 AI 일상화 추진 과정에서 어떤 역할을 담당하며, 어느 과제와 가장 밀접하게 관련되어 있나요?",  # 특정 주체와 역할
        "보고서에서 말하는 '생성형 AI 선도국가 도약'의 의미는 무엇이며, 이를 위한 핵심 전략은 무엇인가요?",  # 개념 및 전략 요약
    ]

    print("\n--- 하이브리드 (Vector + Graph) 쿼리 엔진 테스트 ---")
    for i, q in enumerate(questions_hybrid_rag):
        print(f"\n[질문 {i + 1}] {q}")
        response = hybrid_query_engine.query(q)
        print(f"  [답변] {response.response}")
        # print(f"  [소스 노드] {response.source_nodes}")  # 참조 노드 확인 가능
else:
    print("문서 또는 지식 그래프 인덱스가 생성되지 않아 하이브리드 검색을 수행할 수 없습니다.")

/home/downtown/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



하이브리드 (Vector + Graph) 쿼리 엔진이 준비되었습니다.

--- 하이브리드 (Vector + Graph) 쿼리 엔진 테스트 ---

[질문 1] 전국민 AI 일상화를 위한 핵심 추진 과제는 무엇이며, 각 과제와 관련된 주요 서비스나 사업은 무엇인가요?
  [답변] 전국민 AI 일상화를 위한 핵심 추진 과제는 다음과 같습니다:

1. **AI 문해력 제고**: 교수학습자료 개발 및 인정과목 개설을 통해 AI 리터러시를 함양합니다. 이를 통해 누구나 일상과 일터에서 AI를 활용할 수 있도록 지원합니다.

2. **디지털 튜터 배치 및 디지털 문제해결센터 운영**: 교육부에서 디지털 튜터를 배치하고 문제해결센터를 운영하여 AI 활용 능력을 높입니다.

3. **AI 선도학교 운영 및 신규 AI 교육 콘텐츠 제공**: AI 교육을 선도하는 학교를 운영하고, 새로운 교육 콘텐츠를 제공하여 학생들의 AI 활용 능력을 강화합니다.

4. **소상공인 및 중소기업 AI 활용 지원**: 소상공인 점포에 AI 융합 디지털 기기를 도입하고, 서비스업 매장에 AI 제품 및 서비스를 지원하여 산업 전반의 AI 내재화를 촉진합니다.

5. **AI 기반 공공서비스 혁신**: 행정 서비스 및 민생 현안 해결을 위한 AI 도입을 통해 정부 업무 혁신과 국민 편익 증대를 목표로 합니다.

이와 관련된 주요 서비스나 사업으로는 AI 홈 제품 및 서비스, 디지털 교과서, AI 보조기기, AI 큐레이터 도입, 그리고 AI를 활용한 건강관리 및 돌봄 서비스 등이 있습니다. 이러한 사업들은 AI의 일상화와 대중화를 가속화하는 데 기여하고 있습니다.

[질문 2] 보고서에 언급된 '사회적 약자 지원'과 관련된 AI 서비스들은 서로 어떻게 연관되어 있으며, 어떤 부처들이 협력하여 추진하나요?
  [답변] '사회적 약자 지원'과 관련된 AI 서비스들은 독거노인, 보호아동, 장애인 등 다양한 대상에 맞춰 건강관리, 학습 지원, 재활 운동 등을 제공하는 방식으로 연관되어 있습니다. 예를 들어

## 마무리

이 실습을 통해 비정형 텍스트에서 지식 그래프를 추출하고, 이를 활용하여 관계 기반 질문에 답변하는 Graph RAG를 구현하는 방법을 익혔습니다.  
또한, Vector RAG와 Graph RAG를 결합한 하이브리드 검색의 가능성과 함께, **Neo4j 그래프 데이터베이스 연동**에 대해서도 살펴보았습니다.  